## Pull OptionMetrics Data from WRDS

### Imports

In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pathlib import Path 


import pandas as pd
import numpy as np
import wrds

import config


OUTPUT_DIR = Path(config.OUTPUT_DIR)
DATA_DIR = Path(config.DATA_DIR)
WRDS_USERNAME = config.WRDS_USERNAME
START_DATE = config.START_DATE
END_DATE = config.END_DATE
db = wrds.Connection(wrds_username=WRDS_USERNAME)

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


### 1. Description of OptionMetrics

#### OptionMetrics is a comprehensive database of historical option price, underlying security information, implied volatility, and sensitivity information for the entire US listed and equity options markets from 1996 till present. The link below contains the specific description:
https://wrds-www.wharton.upenn.edu/pages/support/manuals-and-overviews/optionmetrics/wrds-overview-optionmetrics/#option-and-underlying-security-information-datasets

The link below includes all tables of OptionMetrics database, we can find the table we need from the list:
https://wrds-www.wharton.upenn.edu/data-dictionary/optionm_all/

### 2. Pull option price data:
The Option Price files (opprcdYYYY) contain historical price, implied volatility, and sensitivity information for traded options in US option markets. Since the table is sparate for each year, we designed a loop to pull all the data from 1996 to 2012.

https://wrds-www.wharton.upenn.edu/data-dictionary/optionm_all/opprcd1996/

The link above contains details of 1996 option price table, such as variable names and postgres schema. We need tables from 1996 untile 2012.

In [5]:
def pull_Option_price(start_date=START_DATE, end_date=END_DATE): 
    df = []
    year_list = [1996, 1997, 1998, 1999, 
             2000, 2001, 2002, 2003, 
             2004, 2005, 2006, 2007, 
             2008, 2009, 2010, 2011, 2012]  # the list of years
    
    for year in year_list:
    
        query = f"""
            SELECT 
                opp.date, opp.secid, opp.exdate,  
                opp.cp_flag, opp.strike_price, 
                opp.forward_price, opp.impl_volatility, 
                opp.volume, opp.contract_size, 
                opp.best_bid, opp.best_offer
            
            FROM
                optionm_all.opprcd{year} AS opp
            
            WHERE
                opp.secid = 108105
            
            
        """ 
        
        data = db.raw_sql(query, date_cols=["date"])
        df.append(data)
            
    df = pd.concat(df, ignore_index=True)
    df = df[df['date'] <= end_date]
    return df

In [6]:
df = pull_Option_price()
df

,date,secid,exdate,cp_flag,strike_price,forward_price,impl_volatility,volume,contract_size,best_bid,best_offer
0,1996-01-05,108105.0,1996-03-16,C,525000.0,None,NaN,0.0,100.0,92.875,93.875
1,1996-01-05,108105.0,1996-03-16,C,550000.0,None,NaN,0.0,100.0,68.750,69.750
2,1996-01-09,108105.0,1996-02-17,C,570000.0,None,NaN,0.0,100.0,38.500,39.500
3,1996-01-09,108105.0,1996-01-20,C,580000.0,None,NaN,0.0,100.0,25.625,26.625
4,1996-01-17,108105.0,1996-01-20,C,595000.0,None,NaN,80.0,100.0,10.750,11.250
...,...,...,...,...,...,...,...,...,...,...,...
3410578,2012-01-31,108105.0,2012-02-03,P,1395000.0,None,NaN,0.0,100.0,80.800,83.500
3410579,2012-01-31,108105.0,2012-02-03,P,1400000.0,None,NaN,10.0,100.0,85.600,88.500
3410580,2012-01-31,108105.0,2012-02-03,P,1425000.0,None,NaN,0.0,100.0,111.200,113.700
3410581,2012-01-31,108105.0,2012-02-03,P,1450000.0,None,NaN,0.0,100.0,135.600,138.700


### 3. Pull Security Price Data
The Security Price files (secprdYYYY) contain the price history for the security for the corresponding calendar year.
(https://wrds-www.wharton.upenn.edu/data-dictionary/optionm_all/secprd1996/)

Similarly, we pull the data from 1996 until 2012 with a loop

In [7]:
def pull_Security_price(end_date=END_DATE):
    df = []
    year_list = [1996, 1997, 1998, 1999, 
             2000, 2001, 2002, 2003, 
             2004, 2005, 2006, 2007, 
             2008, 2009, 2010, 2011, 2012]
    
    for year in year_list:
    
        query = f"""
            SELECT
                sec.date, sec.secid, sec.open, sec.close

            FROM
                optionm_all.secprd{year} AS sec

            WHERE
                sec.secid = 108105

        """
    
        data = db.raw_sql(query, date_cols=["date"])
        df.append(data)
        
    df = pd.concat(df, ignore_index=True)
    df = df[df['date'] <= end_date] 
    
    return df

In [8]:
df = pull_Security_price()
df

,date,secid,open,close
0,1996-01-02,108105.0,615.93,620.73
1,1996-01-03,108105.0,620.73,621.32
2,1996-01-04,108105.0,621.32,617.70
3,1996-01-05,108105.0,617.70,616.71
4,1996-01-08,108105.0,616.71,618.46
...,...,...,...,...
4045,2012-01-25,108105.0,1314.40,1326.06
4046,2012-01-26,108105.0,1326.28,1318.43
4047,2012-01-27,108105.0,1318.25,1316.33
4048,2012-01-30,108105.0,1316.16,1313.01


### 4. Pull Daily Interest Rate Data 
According to the paper, we need T-bill interest rates data, Federal Reserve’s H.15 release.
https://wrds-www.wharton.upenn.edu/data-dictionary/frb_all/rates_daily/

Considering the maturity of options are 30, 60 and 90 days, we select T-Bills Secondary Market 3-month as T-bill rates.

In [9]:
def pull_TB_int_rate(wrds_username=WRDS_USERNAME, start_date="1995-12-04", end_date=END_DATE):
    query = f"""
		SELECT
			rd.date, rd.tb_m3

		FROM
			frb_all.rates_daily AS rd
		
  		WHERE
			rd.date >= '{start_date}' AND 
        	rd.date <= '{end_date}'
        
    """
    
    df = db.raw_sql(query, date_cols=["date"]).set_index('date')
    
    df = df.loc['1996-01-04':]
    df = df.drop(df.index[0])
    df = df.reset_index().rename(columns={'index': 'date'})
    
    return df 

In [11]:
df = pull_TB_int_rate()
df

,date,tb_m3
0,1996-01-04,5.04
1,1996-01-05,5.03
2,1996-01-08,5.03
3,1996-01-09,5.01
4,1996-01-10,5.04
...,...,...
4188,2012-01-25,0.04
4189,2012-01-26,0.05
4190,2012-01-27,0.06
4191,2012-01-30,0.05


### 5. Merge all the data
Now, we have all the data we need, and we merge them into one DataFrame.

In [12]:
def merge_data():
    
    df1 = pull_Option_price()
    df2 = pull_Security_price()
    df3 = pull_TB_int_rate()
    
    df = pd.merge(df1, df2, on=['date', 'secid'])
    df = pd.merge(df, df3, on=['date'])
    
    return df

In [13]:
df = merge_data()
df

,date,secid,exdate,cp_flag,strike_price,forward_price,impl_volatility,volume,contract_size,best_bid,best_offer,open,close,tb_m3
0,1996-01-05,108105.0,1996-03-16,C,525000.0,None,NaN,0.0,100.0,92.8750,93.875,617.70,616.71,5.03
1,1996-01-05,108105.0,1996-03-16,C,550000.0,None,NaN,0.0,100.0,68.7500,69.750,617.70,616.71,5.03
2,1996-01-05,108105.0,1996-02-17,C,570000.0,None,0.116110,0.0,100.0,48.2500,49.250,617.70,616.71,5.03
3,1996-01-05,108105.0,1996-03-16,P,550000.0,None,0.179154,623.0,100.0,1.0000,1.375,617.70,616.71,5.03
4,1996-01-05,108105.0,1996-01-20,P,525000.0,None,0.353404,100.0,100.0,0.0625,0.125,617.70,616.71,5.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3410575,2012-01-31,108105.0,2012-02-03,P,1395000.0,None,NaN,0.0,100.0,80.8000,83.500,1313.53,1312.41,0.06
3410576,2012-01-31,108105.0,2012-02-03,P,1400000.0,None,NaN,10.0,100.0,85.6000,88.500,1313.53,1312.41,0.06
3410577,2012-01-31,108105.0,2012-02-03,P,1425000.0,None,NaN,0.0,100.0,111.2000,113.700,1313.53,1312.41,0.06
3410578,2012-01-31,108105.0,2012-02-03,P,1450000.0,None,NaN,0.0,100.0,135.6000,138.700,1313.53,1312.41,0.06


In [14]:
db.close()

### From the Dataframe above, we can find the lenth is close to the Table B1.